<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/prophet-code/prophet-random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 骆驼先知

骆驼先知是 李鲁鲁 受到吴恩达的prompt Engineering的启发

制作的一个纪伯伦的《先知》的拓展版本

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

In [ ]:
! pip install openai

In [16]:
import openai
import os

# 导入第三方库

openai.api_key  = 'sk-'
# 李鲁鲁 在这里设置你的API_KEY

In [3]:
#从项目中获取数据
!git clone https://github.com/LC1332/Prophet-Andrew-Ng

Cloning into 'Prophet-Andrew-Ng'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 224 (delta 13), reused 19 (delta 7), pack-reused 189
Receiving objects: 100% (224/224), 666.25 KiB | 15.49 MiB/s, done.
Resolving deltas: 100% (111/111), done.


## 数据读取

读取prompt-data中的文本数据，作为in-context-learning的数据

In [4]:
# 如果你使用本地的版本，你的路径应该为
# prophet_data_folder = './../prophet-data'

# 在这里我们考虑colab的版本
prophet_data_folder = '/content/Prophet-Andrew-Ng/prophet-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(title_to_text[title]))

婚姻 292
爱 789
自由 781
孩子 315
劳作 935


我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [5]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


在这里我们先假设一个query的主题 "时间"

In [6]:
query_topic = "时间"

定义先知和市民的身份，这里是为了方便后面的泛化

In [7]:
prophet_name = "先知"

citizen_name = "市民"

形成一个shot的句子输出

In [8]:
# 实现一个Python函数 ensemble_one_shot ，输入为prophet_name, citizen_name, sample_topic，sample_answer， 输出为一个组织好的字符串
# 例子输入: 先知, 市民, 衣服， "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。 \n 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"
# 例子输出: """
# <市民>:请和我们讨论一下"衣服"

# <先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
# 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
# """
def ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer):
    # 组织对话文本
    dialogue = "<{}>:```请和我们讨论一下\"{}\"```\n\n<{}>:```{}```\n\n".format(citizen_name, sample_topic, prophet_name, sample_answer)
    return dialogue

# unit test for ensemble_one_shot
prophet_name = "先知"
citizen_name = "市民"
sample_topic = "衣服"
sample_answer = "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。\n尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"

dialogue = ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)
print(dialogue)

<市民>:```请和我们讨论一下"衣服"```

<先知>:```你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。```




下面我们来组织完整的prompt，我们假设已经选取了两个主题作为例子 孩子和爱

In [ ]:
selected_sample = ["孩子","爱"]

def organize_prompt( query_topic, selected_sample ):
    prompt = """你的任务是以一致的风格回答问题。\n\n"""
    
    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_answer = title_to_text[sample_topic]
        prompt += ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)

    prompt += """<{}>:```请和我们讨论一下"{}"```\n\n""".format(citizen_name, query_topic)

    return prompt

# write a unit test for organize_prompt
query_topic = "时间"
selected_sample = ["孩子","爱"]
prompt = organize_prompt( query_topic, selected_sample )
print(prompt)

在第一个版本中，我们selected_sample先使用随机的策略

在后面的版本中我准备引入LuotuoBERT 去选取更接近主题的问题

In [10]:
import random

# def function random_select_title, random pick k sample from titles
def random_select_title(titles, k):
    return random.sample(titles, k)

# unit test for random_select_title
selected_sample = random_select_title(titles, 2)
print(selected_sample)

['自由', '孩子']


集成所有的代码！

In [17]:
query_topic = "时间"
selected_sample = random_select_title(titles, 2)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

<先知>:```时间是一条河流，它不停地流淌，无法逆转。
我们无法改变时间的流逝，但我们可以改变我们对时间的看法和利用方式。
时间是一种珍贵的资源，我们应该珍惜它，充分利用它来实现我们的目标和梦想。
我们不能让时间虚度而过，而应该把握每一刻，让时间成为我们的朋友和盟友。
同时，我们也应该学会放下过去，珍惜现在，展望未来。
时间是一种无形的财富，它的价值不在于它的长度，而在于我们如何利用它。
让我们珍惜时间，让它成为我们生命中最宝贵的财富之一。```


当然你也可以指定特定的title进行讨论

In [18]:
query_topic = "内卷"
selected_sample = ["劳作","自由"]
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

<先知>:```内卷是一种社会现象，它是指在竞争激烈的环境下，个体为了获得更多的资源和机会，不断地自我压榨和竞争，最终导致整个社会的资源和机会都被消耗殆尽的现象。

内卷是一种不可持续的发展模式，它会导致社会的资源和机会不断减少，同时也会导致人们的身心健康受到损害。
内卷的根源在于社会的不公平和竞争激烈，如果我们能够建立一个更加公平和和谐的社会，就能够有效地遏制内卷现象的发生。

我们需要摆脱内卷的思维模式，不要一味地追求更多的资源和机会，而是要注重自身的内在修养和发展，追求更加有意义和有价值的生活。

同时，我们也需要建立一个更加公平和和谐的社会，让每个人都能够享有平等的机会和资源，从而减少竞争和内卷的发生。只有这样，我们才能够建立一个更加美好和可持续的社会。```


后续

+ 增加更好的前置提示词（提前总结先知的文风）

+ 补充完整的prophet data（一共20+个，现在只有5个）

+ 使用luotuoBERT索引更相关的主题

+ 做一个gradio的前端